# 01 — Connect and Explore

This notebook demonstrates connecting to a brokerage provider and exploring available option chains. The session connects to paper or live trading based on your `config/config.yaml` settings (`is_paper: true` for paper, `false` for live).

## Prerequisites

1. A brokerage OAuth application (e.g., create at https://developer.tastytrade.com for TastyTrade)
2. Copy `.env.example` to `.env` at the project root and fill in your credentials:
   ```bash
   TASTYTRADE_CLIENT_SECRET=your_client_secret
   TASTYTRADE_REFRESH_TOKEN=your_refresh_token
   ```
3. The `load_config()` function auto-discovers `.env` and `config/config.yaml` from the project root.

In [1]:
%load_ext autoreload
%autoreload 2

## 1. Setup & Config

Load configuration from YAML and display settings (credentials are redacted via `SecretStr`).

In [2]:
from options_analyzer.config import load_config

config = load_config()

print(f"Provider: {config.provider.name}")
print(f"Paper trading: {config.provider.is_paper}")
print(f"Client secret: {config.provider.client_secret}")

Provider: tastytrade
Paper trading: False
Client secret: **********


## 2. Connect

Create a session and authenticate using the provider factory. The factory reads `config.provider.name` and returns the appropriate port implementations.

In [ ]:
from options_analyzer.factory import create_providers

providers = await create_providers(config)
market_data = providers.market_data
account_provider = providers.account

# Verify connection by listing accounts
accounts = await account_provider.get_accounts()
print(f"Connected to {providers.provider_name}! Accounts: {accounts}")

## 3. Fetch Option Chain

Retrieve the full option chain for a liquid underlying. The chain is organized by expiration date.

In [4]:
UNDERLYING = "SPX"

chain = await market_data.get_option_chain(UNDERLYING)

print(f"\n{UNDERLYING} Option Chain")
print(f"{'='*40}")
print(f"Expirations available: {len(chain)}")
print()
for exp_date in sorted(chain.keys())[:10]:
    n_contracts = len(chain[exp_date])
    print(f"  {exp_date}  —  {n_contracts} contracts")

if len(chain) > 10:
    print(f"  ... and {len(chain) - 10} more expirations")


SPX Option Chain
Expirations available: 56

  2026-02-10  —  460 contracts
  2026-02-11  —  470 contracts
  2026-02-12  —  470 contracts
  2026-02-13  —  538 contracts
  2026-02-17  —  426 contracts
  2026-02-18  —  404 contracts
  2026-02-19  —  404 contracts
  2026-02-20  —  1816 contracts
  2026-02-23  —  372 contracts
  2026-02-24  —  356 contracts
  ... and 46 more expirations


## 4. Explore Chain Data

Filter to a specific expiration, display calls and puts, and identify the ATM strike.

In [5]:
from decimal import Decimal

# Get underlying price
spot_price = await market_data.get_underlying_price(UNDERLYING)
print(f"{UNDERLYING} current mid price: ${spot_price:.2f}")

# Pick the nearest monthly expiration (skip weeklies — pick one with many strikes)
sorted_exps = sorted(chain.keys())
target_exp = max(sorted_exps[:5], key=lambda d: len(chain[d]))
contracts = chain[target_exp]

print(f"\nSelected expiration: {target_exp}")
print(f"Total contracts: {len(contracts)}")

# Separate calls and puts
from options_analyzer.domain.enums import OptionType

calls = sorted(
    [c for c in contracts if c.option_type == OptionType.CALL],
    key=lambda c: c.strike,
)
puts = sorted(
    [c for c in contracts if c.option_type == OptionType.PUT],
    key=lambda c: c.strike,
)

print(f"Calls: {len(calls)}, Puts: {len(puts)}")

# Find ATM strike
atm_strike = min(calls, key=lambda c: abs(c.strike - spot_price)).strike
print(f"ATM strike: {atm_strike}")

SPX current mid price: $6977.12

Selected expiration: 2026-02-13
Total contracts: 538
Calls: 269, Puts: 269
ATM strike: 6975.0


## 5. Display as DataFrame

Convert option contracts near ATM to a table for easy viewing.

In [6]:
# Show contracts near ATM (± 10 strikes)
near_atm = [
    c for c in contracts
    if abs(c.strike - atm_strike) <= Decimal("10")
]
near_atm.sort(key=lambda c: (c.strike, c.option_type))

# Build a simple table
print(f"\n{'Strike':>8}  {'Type':>5}  {'Symbol':<35}  {'Style':<10}  {'Mult':>4}")
print(f"{'─'*8}  {'─'*5}  {'─'*35}  {'─'*10}  {'─'*4}")
for c in near_atm:
    marker = " ◄ ATM" if c.strike == atm_strike else ""
    print(
        f"{c.strike:>8}  {c.option_type.value:>5}  {c.symbol:<35}  "
        f"{c.exercise_style.value:<10}  {c.multiplier:>4}{marker}"
    )


  Strike   Type  Symbol                               Style       Mult
────────  ─────  ───────────────────────────────────  ──────────  ────
  6965.0   call  SPXW  260213C06965000                european     100
  6965.0    put  SPXW  260213P06965000                european     100
  6970.0   call  SPXW  260213C06970000                european     100
  6970.0    put  SPXW  260213P06970000                european     100
  6975.0   call  SPXW  260213C06975000                european     100 ◄ ATM
  6975.0    put  SPXW  260213P06975000                european     100 ◄ ATM
  6980.0   call  SPXW  260213C06980000                european     100
  6980.0    put  SPXW  260213P06980000                european     100
  6985.0   call  SPXW  260213C06985000                european     100
  6985.0    put  SPXW  260213P06985000                european     100


## 6. Cleanup

Disconnect the session to release resources.

In [ ]:
await providers.disconnect()
print("Session disconnected.")